In [1]:
import pyfdb 
import emohawk

<div class="alert alert-block alert-warning">
To run this scirpt FDB access is needed and the <b>FDB_HOME</b> environment variable has to be set correctly.
</div>

In [2]:
# date must be adjusted since FDB only stores recent dates
request = {
    'class': 'od',
    'expver': '0001',
    'stream': 'oper',
    'date': '20230402',
    'time': [0, 12],
    'domain': 'g',
    'type': 'an',
    'levtype': 'sfc',
    'step': 0,
    'param': [151, 167]
}

# Must be set correctly
%env FDB_HOME=/home/fdbprod

fdb = pyfdb.FDB()

env: FDB_HOME=/home/fdbprod


#### Iteration with one field at a time in memory

In [3]:
r = fdb.retrieve(request)

fs = emohawk.from_source("stream", r)
type(fs)

emohawk.sources.stream.StreamSingleIterSource

Nothing is read at this moment.

We can only use *fs* for iteration. Fields crerated in the iteration get deleted when going out of scope:

In [4]:
for f in fs:
    print(f"type(f)={type(f)}")
    print(f"  path={f.path} offset={f.offset}")
    print(f"  param={f['param']}")
    print(f"  shape={f.values.shape} mean={f.values.mean()}")

type(f)=<class 'emohawk.readers.grib.memory.GribFieldInMemory'>
  path=None offset=None
  param=msl
  shape=(6599680,) mean=101191.5242918596
type(f)=<class 'emohawk.readers.grib.memory.GribFieldInMemory'>
  path=None offset=None
  param=2t
  shape=(6599680,) mean=287.2112324542274


Once the iteration is completed, there is nothing left in fs.

In [5]:
for f in fs:
    print(f"type(f)={type(f)}")

#### Storing all the fields in memory

In [6]:
r = fdb.retrieve(request)

fs = emohawk.from_source("stream", r, group_by=0)
type(fs)

emohawk.sources.stream.StreamSource

Nothing is read at this moment:

In [7]:
print(f"stored fields count={len(fs._reader._fields)}")

stored fields count=0


If we call any function on the fieldlist it reads the messages into memory

In [8]:
len(fs)

2

In [9]:
print(f"stored fields count={len(fs._reader._fields)}")

stored fields count=2


In [10]:
fs.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,msl,surface,0,20230217,0,0,an,0,reduced_gg
1,ecmf,2t,surface,0,20230217,0,0,an,0,reduced_gg


In [11]:
a = fs.sel(param="2t")
print(type(a))

<class 'emohawk.readers.grib.index.MaskFieldSet'>


In [12]:
a.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,2t,surface,0,20230217,0,0,an,0,reduced_gg


In [13]:
fs.to_xarray()

<xarray.Dataset>
Dimensions:     (number: 1, time: 1, step: 1, surface: 1, values: 6599680)
Coordinates:
  * number      (number) int64 0
  * time        (time) datetime64[ns] 2023-02-17
  * step        (step) timedelta64[ns] 00:00:00
  * surface     (surface) float64 0.0
    latitude    (values) float64 ...
    longitude   (values) float64 ...
    valid_time  (time, step) datetime64[ns] ...
Dimensions without coordinates: values
Data variables:
    msl         (number, time, step, surface, values) float32 ...
    t2m         (number, time, step, surface, values) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-02-18T17:48 GRIB to CDM+CF via cfgrib-0.9.1...